In [1]:
import pandas as pd
from pathlib import Path
import re

In [2]:
# --- 1. Где лежат ваши файлы -----------------------------------------------
DATA_DIR = Path("../new_data/datasets_circle_all")      # при необходимости поменяйте путь
PATTERN   = "dataset_raw*.csv"       

def natural_key(p):                     # сортировка raw1, raw2, …
    parts = re.split(r"(\d+)", p.stem)
    return [int(s) if s.isdigit() else s for s in parts]

dfs_circle = []
for f in sorted(DATA_DIR.glob(PATTERN), key=natural_key):
    df = pd.read_csv(
        f,
        engine="python"     # терпимее к кривым строкам
    )
    dfs_circle.append(df[1:])

In [3]:
dfs_circle[0]

,t,ax,ay,yawRate,steer,gas,t_sec,yaw,yaw_rad,ax_world,ay_world,angular_accel,x_world,y_world
1,1946.0,-0.0253,0.0321,-0.4268,3540.0,2922.0,1.946,-0.076397,-0.001333,-0.025257,0.032134,-13.284916,-0.000809,0.001030
2,2666.0,-0.0480,-0.0012,0.0915,2984.0,2874.0,2.666,-0.010517,-0.000184,-0.048000,-0.001191,0.719861,-0.028948,0.004553
3,3386.0,-0.0499,0.0268,-0.1220,3770.0,2886.0,3.386,-0.098357,-0.001717,-0.049854,0.026886,-0.296528,-0.082930,0.022015
4,4106.0,-0.0639,-0.0111,0.0915,3078.0,3028.0,4.106,-0.032477,-0.000567,-0.063906,-0.011064,0.296528,-0.170042,0.033741
5,4826.0,-0.0664,0.0194,-0.3049,3272.0,3460.0,4.826,-0.252005,-0.004398,-0.066314,0.019692,-0.550556,-0.291531,0.055675
6,5545.0,-0.0474,0.0404,-0.2134,3568.0,3660.0,5.545,-0.405440,-0.007076,-0.047113,0.040734,0.127260,-0.437207,0.098637
7,6248.0,-0.0336,0.0644,-0.1220,3706.0,3982.0,6.248,-0.491206,-0.008573,-0.033047,0.064686,0.130014,-0.595973,0.172611
8,6607.0,-0.4937,-0.6208,72.9268,3706.0,3988.0,6.607,25.689515,0.448367,-0.175788,-0.773454,203.478552,-0.699705,0.110704
9,6788.0,-0.4745,-3.1374,109.3598,3776.0,3988.0,6.788,45.483639,0.793839,1.904446,-2.538014,201.287293,-0.689613,-0.003657
10,6968.0,-0.9571,-4.0245,150.9451,3846.0,4002.0,6.968,72.653757,1.268047,3.556113,-2.113458,231.029444,-0.564359,-0.185861


In [4]:
import pandas as pd
from pathlib import Path
import re

# --- 1. Где лежат ваши файлы -----------------------------------------------
DATA_DIR = Path("../new_data/datasets_circle_all")   # при необходимости поменяйте путь
PATTERN = "dataset_raw*.csv"       

def natural_key(p):  # сортировка raw1, raw2, …
    parts = re.split(r"(\d+)", p.stem)
    return [int(s) if s.isdigit() else s for s in parts]

dfs_circle = []

for f in sorted(DATA_DIR.glob(PATTERN), key=natural_key):
    df = pd.read_csv(f, engine="python")
    df = df[1:].copy()  # пропускаем первую строку (возможно, повреждена)

    # --- 2. Добавим столбец "is_drift" ---------------------------------------
    YAW_THR   = 150     # |°/с|
    AY_THR    =  2.5    # |g|   (~25 м/с²)
    ANGACC_THR= 100     # |°/с²| -- для отлова самого начала
    
    df["is_drift"] = (
            (df["yawRate"].abs()    > YAW_THR) |
            (df["ay_world"].abs()   > AY_THR ) &
            (df["angular_accel"].abs() > ANGACC_THR)
    ).astype(int)

    dfs_circle.append(df)

In [5]:
dfs_circle[1]

,t,ax,ay,yawRate,steer,gas,t_sec,yaw,yaw_rad,ax_world,ay_world,angular_accel,x_world,y_world,is_drift
1,3156.0,-0.0650,0.0152,0.1829,4002.0,4002.0,3.156,0.131688,0.002298,-0.065035,0.015051,0.296389,-0.033714,0.007802,0
2,3228.0,-0.0740,0.0294,-0.0305,4002.0,4014.0,3.228,0.129492,0.002260,-0.074066,0.029233,-2.963889,-0.037469,0.008734,0
3,3948.0,-1.5245,-0.8968,328.4451,4004.0,4002.0,3.948,236.609964,4.129623,0.090209,1.766412,456.216111,-0.028259,0.933760,1
4,4668.0,-2.4583,-1.3433,236.9817,4002.0,4002.0,4.668,407.236788,7.107623,-0.682907,-2.716861,-127.032500,-0.373067,0.450365,1
5,4847.0,-2.6164,-2.1075,235.6707,4002.0,4002.0,4.847,449.421843,7.843891,2.080992,-2.637533,-7.324022,-0.392113,0.245679,1
6,5027.0,-3.0202,-2.0940,269.9085,4004.0,4002.0,5.027,498.005373,8.691833,3.645649,-0.464421,190.210000,-0.293146,0.024801,1
7,5747.0,-3.0339,-1.3390,260.4573,4004.0,4002.0,5.747,685.534629,11.964836,-3.259105,0.612945,-13.126667,-1.586800,-0.540957,1
8,6107.0,-2.7546,-1.8714,260.2744,4002.0,4002.0,6.107,779.233413,13.600189,0.198923,-3.324213,-0.508056,-2.207847,-1.254654,1
9,6287.0,-3.2002,-2.6765,262.3171,4004.0,4002.0,6.287,826.450491,14.424282,3.473191,-2.311250,11.348333,-2.405839,-1.686387,1
10,7007.0,-3.3497,-2.4084,260.0915,4002.0,4002.0,7.007,1013.716371,17.692688,-3.552286,2.098129,-3.091111,-5.039312,-2.325649,1


In [32]:
# --- 1. Где лежат ваши файлы -----------------------------------------------
DATA_DIR = Path("../data_in_world_frame/data_in_world_frame/datasets_slide")  # при необходимости поменяйте путь
PATTERN = "dataset_raw*.csv"       

def natural_key(p):  # сортировка raw1, raw2, …
    parts = re.split(r"(\d+)", p.stem)
    return [int(s) if s.isdigit() else s for s in parts]

dfs_slide = []

for f in sorted(DATA_DIR.glob(PATTERN), key=natural_key):
    df = pd.read_csv(f, engine="python")
    df = df[1:].copy()  # пропускаем первую строку (возможно, повреждена)

    # --- 2. Добавим столбец "is_drift" ---------------------------------------
    YAW_THR   = 150     # |°/с|
    AY_THR    =  2.5    # |g|   (~25 м/с²)
    ANGACC_THR= 100     # |°/с²| -- для отлова самого начала
    
    df["is_drift"] = (
            (df["yawRate"].abs()    > YAW_THR) |
            (df["ay_world"].abs()   > AY_THR ) &
            (df["angular_accel"].abs() > ANGACC_THR)
    ).astype(int)

    dfs_slide.append(df)

In [33]:
dfs_slide[0]

,t,ax,ay,yawRate,steer,t_sec,yaw,yaw_rad,ax_world,ay_world,angular_accel,is_drift
1,1101,-0.0027,0.2319,0.4878,3394,1.101,0.025853,0.000451,-0.002805,0.231899,0.575472,0
2,1156,-0.0510,0.2602,3.0793,3742,1.156,0.195215,0.003407,-0.051886,0.260025,47.118182,0
3,1210,-0.1069,0.2608,9.5732,3906,1.210,0.712168,0.012430,-0.110133,0.259451,120.257407,0
4,1263,-0.1343,0.3202,11.4024,3996,1.263,1.316495,0.022977,-0.141621,0.317030,34.513208,0
5,1317,-0.1735,0.3644,11.1585,4004,1.317,1.919054,0.033494,-0.185606,0.358386,-4.516667,0
...,...,...,...,...,...,...,...,...,...,...,...,...
72,4934,-1.7186,0.4560,9.7866,3164,4.934,255.778200,4.464172,0.864244,1.553900,285.681481,0
73,4988,-1.3710,0.6770,-4.8780,3004,4.988,255.514788,4.459574,0.998408,1.158081,-271.566667,0
74,5043,-1.1658,0.8753,-4.9085,2980,5.043,255.244820,4.454863,1.143352,0.904426,-0.554545,0
75,5097,-1.1673,0.9047,-0.9756,2990,5.097,255.192138,4.453943,1.172990,0.897310,72.831481,0


In [30]:
# --- 1. Где лежат ваши файлы -----------------------------------------------
DATA_DIR = Path("../data_in_world_frame/data_in_world_frame/datasets_straight")  # при необходимости поменяйте путь
PATTERN = "dataset_raw*.csv"       

def natural_key(p):  # сортировка raw1, raw2, …
    parts = re.split(r"(\d+)", p.stem)
    return [int(s) if s.isdigit() else s for s in parts]

dfs_straight = []

for f in sorted(DATA_DIR.glob(PATTERN), key=natural_key):
    df = pd.read_csv(f, engine="python")
    df = df[1:].copy()  # пропускаем первую строку (возможно, повреждена)

    # --- 2. Добавим столбец "is_drift" ---------------------------------------
    YAW_THR   = 150     # |°/с|
    AY_THR    =  2.5    # |g|   (~25 м/с²)
    ANGACC_THR= 100     # |°/с²| -- для отлова самого начала
    
    df["is_drift"] = (
            (df["yawRate"].abs()    > YAW_THR) |
            (df["ay_world"].abs()   > AY_THR ) &
            (df["angular_accel"].abs() > ANGACC_THR)
    ).astype(int)

    dfs_straight.append(df)

In [37]:
dfs_straight[1]

,t,ax,ay,yawRate,steer,t_sec,yaw,yaw_rad,ax_world,ay_world,angular_accel,is_drift
1,1094,-0.0653,0.2628,0.5793,2978,1.094,0.031282,0.000546,-0.065443,0.262764,1.694444,0
2,1147,-0.0848,0.3668,0.4573,2980,1.147,0.055519,0.000969,-0.085155,0.366718,-2.301887,0
3,1202,-0.1339,0.4342,0.7012,2992,1.202,0.094085,0.001642,-0.134613,0.433980,4.434545,0
4,1256,-0.1641,0.4944,0.4878,2990,1.256,0.120426,0.002102,-0.165139,0.494054,-3.951852,0
5,1310,-0.1878,0.5406,0.5793,2978,1.310,0.151708,0.002648,-0.189231,0.540101,1.694444,0
...,...,...,...,...,...,...,...,...,...,...,...,...
80,5359,-5.4715,0.2636,186.9207,3004,5.359,107.617985,1.878288,1.404818,-5.294647,121.385185,1
81,5413,-5.0744,0.0605,193.3232,3000,5.413,118.057438,2.060491,2.333387,-4.506495,118.564815,1
82,5467,-5.6478,-0.3179,199.3902,2984,5.467,128.824508,2.248412,3.788482,-4.200727,112.351852,1
83,5521,-5.2202,-0.2768,216.4329,2982,5.521,140.511885,2.452395,4.204745,-3.105998,315.605556,1


In [7]:
n = 1
for df_i in dfs_circle:
    df_i.to_csv(f'../new_data/circle/dataset_raw{n}.csv', index=False)
    n += 1

In [41]:
n = 1
for df_i in dfs_slide:
    df_i.to_csv(f'../full_dataset/slide/dataset_raw{n}.csv', index=False)
    n += 1

In [42]:
n = 1
for df_i in dfs_straight:
    df_i.to_csv(f'../full_dataset/straight/dataset_raw{n}.csv', index=False)
    n += 1

In [44]:
dfs_circle[0]

,t,ax,ay,yawRate,steer,t_sec,yaw,yaw_rad,ax_world,ay_world,angular_accel,is_drift
1,1202,0.1118,0.2270,0.2134,2820,1.202,0.034571,0.000603,0.111663,0.227067,-2.070370,0
2,1255,0.0631,0.3109,1.7378,2970,1.255,0.126674,0.002211,0.062412,0.311039,28.762264,0
3,1309,0.1261,0.3698,0.7317,2948,1.309,0.166186,0.002900,0.125027,0.370164,-18.631481,0
4,1363,0.3102,0.5118,-0.0610,2168,1.363,0.162892,0.002843,0.308744,0.512680,-14.679630,0
5,1418,0.2596,0.5095,1.2195,1968,1.418,0.229964,0.004014,0.257553,0.510538,23.281818,0
...,...,...,...,...,...,...,...,...,...,...,...,...
71,4980,2.3273,2.4586,-297.4695,1966,4.980,-605.519244,-10.568304,-3.201977,1.099264,-41.992453,1
72,5035,2.2454,2.3054,-300.8842,1968,5.035,-622.067875,-10.857133,-2.593207,1.905772,-62.085455,1
73,5089,1.4640,2.4443,-295.3964,1968,5.089,-638.019281,-11.135537,-2.216160,1.790679,101.625926,1
74,5143,1.2038,2.6919,-291.0976,1968,5.143,-653.738551,-11.409890,-1.979537,2.185610,79.607407,1


In [9]:
dfs_circle[3]

,t,ax,ay,yawRate,steer,gas,t_sec,yaw,yaw_rad,ax_world,ay_world,angular_accel,x_world,y_world,is_drift
1,2648.0,-0.0586,0.0067,-0.2439,2154.0,3256.0,2.648,-0.052682,-0.000919,-0.058594,0.006754,-0.987963,-0.002734,0.000315,0
2,2828.0,-0.0057,-0.1282,-8.7805,3734.0,3294.0,2.828,-1.633172,-0.028504,-0.009351,-0.127985,-47.425556,-0.005315,-0.003569,0
3,3548.0,-0.0539,-0.0935,0.0610,3896.0,3494.0,3.548,-1.589252,-0.027738,-0.056472,-0.091969,12.279861,-0.044915,-0.066782,0
4,4268.0,-0.0675,-0.0528,-0.2134,3906.0,3710.0,4.268,-1.742900,-0.030419,-0.069075,-0.050723,-0.381111,-0.120323,-0.156290,0
5,4987.0,-0.1115,-0.0647,-0.1524,3684.0,3880.0,4.987,-1.852476,-0.032332,-0.113533,-0.061062,0.084840,-0.254318,-0.277241,0
6,5167.0,-0.1802,0.0225,-0.1829,3942.0,3924.0,5.167,-1.885398,-0.032906,-0.179362,0.028416,-0.169444,-0.293675,-0.306599,0
7,5886.0,0.2327,-0.2641,106.6159,3966.0,4002.0,5.886,74.771434,1.305008,0.315950,0.155158,148.537969,-0.287550,-0.343662,0
8,6049.0,-0.2821,-1.5989,170.3354,4002.0,4002.0,6.049,102.536104,1.789593,1.622012,0.071674,390.917178,-0.243066,-0.350159,1
9,6768.0,-0.2544,-1.8992,78.5976,4004.0,4002.0,6.768,159.047779,2.775907,0.916712,1.682652,-127.590821,0.427058,0.491044,0
10,6948.0,-2.3603,-5.1308,52.3781,4002.0,4002.0,6.948,168.475837,2.940458,3.337755,4.555822,-145.663889,0.702966,0.849246,1
